In [1]:
import odbc
import win32com.client
import pandas as pd
import pyodbc

def connection():
    server = 'tcp:192.168.1.6'
    database = 'planeamiento'
    username = 'sa'
    password = 'sa'

    return pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)


In [2]:
cnxn = connection()
query = 'select * from planeamiento..ctrl_excel where end_processing_date is null'

df = pd.read_sql(query,cnxn)

In [3]:
df

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
0,40,Z:\....Candidatos a borrar\Archivos Viejos\,LINEA 1118 - ORIGINAL (NO TOCAR).xlsm,2020-01-23 12:10:12.430,2020-01-24 16:13:16.420,None,3A8037B6-12FA-47D6-82D5-5C7B1F2F035D,None
1,41,Z:\....Candidatos a borrar\Archivos Viejos\,Observaciones.xlsm,2020-01-23 12:10:12.440,NaT,None,None,None
2,42,Z:\....Candidatos a borrar\Archivos Viejos\TODO\,IMPRESION ROTULOS DEPOSITO.xls,2020-01-23 12:10:12.707,NaT,None,None,None
3,43,Z:\....Candidatos a borrar\Carpetas compartidas\,primeros pedidos por zona 3.xlsx,2020-01-23 12:10:12.833,NaT,None,None,None
4,44,Z:\....Candidatos a borrar\consultas\movimientos\,Faltannu por fecha.xls,2020-01-23 12:10:12.963,NaT,None,None,None
...,...,...,...,...,...,...,...,...
34654,34695,Z:\Varios\,INTERNOS ACTUALIZADOS 2019.xlsx,2020-01-23 13:18:48.013,NaT,None,None,None
34655,34696,Z:\Varios\,stock en negativos 27 de septiembre.xlsx,2020-01-23 13:18:48.083,NaT,None,None,None
34656,34697,Z:\Ventas\,Libro1.xlsx,2020-01-23 13:18:48.230,NaT,None,None,None
34657,34698,Z:\Ventas\,presentacion 3-6 v2.xlsm,2020-01-23 13:18:48.250,NaT,None,None,None


In [9]:
def check_excel(file):
    
    #xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
    #file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'
    xl = win32com.client.Dispatch('Excel.Application')
    try:
        wb = xl.Workbooks.Open(file, None,True)
    except Exception as e:
        #print(e)
        return e , []
        
    datos = []

    for x in wb.Connections:
        for i in x.Ranges:
            try:
                for p in i.ListObject.QueryTable.Parameters:
                    datos.append([x,
                                  x.ODBCConnection.CommandText,
                                  x.ODBCConnection.Connection,
                                  i.Parent.Name,
                                  i.Cells(1,1)[2],
                                  p.SourceRange.Address,
                                  p.SourceRange.Parent.Name,
                                  p.SourceRange.Value])

            except Exception as e:
                try:
                    datos.append([x,
                                  x.ODBCConnection.CommandText,
                                  x.ODBCConnection.Connection,
                                  i.Parent.Name,
                                  i.Cells(1,1)[2],
                                  None,
                                  None,
                                  None])
                except:
                    try:
                        for p in i.ListObject.QueryTable.Parameters:
                            datos.append([x,
                                          x.OLEDBConnection.CommandText,
                                          x.OLEDBConnection.Connection,
                                          i.Parent.Name,
                                          i.Cells(1,1)[2],
                                          p.SourceRange.Address,
                                          p.SourceRange.Parent.Name,
                                          p.SourceRange.Value])
                    except:
                        datos.append([x,
                                      x.OLEDBConnection.CommandText,
                                      x.OLEDBConnection.Connection,
                                      i.Parent.Name,
                                      i.Cells(1,1)[2],
                                      None,
                                      None,
                                      None])
                        
                    

    #df = pd.DataFrame(datos)
    #df = df.applymap(str)
    #datos = map(str,datos)
    if type(datos[0]) == list:
        datos = [list(map(str,i))for i in datos]
    else:
        datos = list(map(str,datos))
        
    wb.Close(0)
    xl.Application.Quit()
    
    #return df
    return 0, datos

In [ ]:
xl = win32com.client.DispatchEx("Excel.Application")
wb = xl.workbooks.open("C:\file.xlsm")
xl.run("file.xlsm!macro")
xl.Visible = True
wb.Close(savechanges=1)
xl.Quit()

In [5]:
def insert_file_connection(EXCEL_ID,NAME,STR_CONN,SQL):
    
    aux_cnn = connection()
    
    cursor = aux_cnn.cursor()
    
    params = EXCEL_ID,NAME,STR_CONN,SQL
    
    #query = "EXEC planeamiento..ctrl_excel_InsertFileConnection @excel_id = %s, @name = '%s', @str_conn = '%s', @sql = '%s'"%params
    query = """EXEC planeamiento..ctrl_excel_InsertFileConnection  %s, '%s', '%s', '%s'"""%params
    #sql = "{CALL planeamiento..ctrl_excel_InsertFileConnection (?,?,?,?)}"
    
    try:
        #cursor.execute(sql, params)
        cursor.execute(query)
        val = cursor.fetchval()
        cursor.commit()
        
        
        
    except Exception as e:
        print('*'*100)
        print(e)
        
    
    aux_cnn.close()
    return int(val)
    
    
    

In [27]:
x = insert_file_connection(5,'PROBANDO NAME','CONEXION PROBANDO','SELECT * FROM PRUEBA')
#exec planeamiento..ctrl_excel_InsertFileConnection 5,'PROBANDO NAME','CONEXION PROBANDO','SELECT * FROM PRUEBA'

In [ ]:
x

In [31]:
df.head()

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
0,40,Z:\....Candidatos a borrar\Archivos Viejos\,LINEA 1118 - ORIGINAL (NO TOCAR).xlsm,2020-01-23 12:10:12.430,2020-01-24 16:13:16.420,None,3A8037B6-12FA-47D6-82D5-5C7B1F2F035D,None
1,41,Z:\....Candidatos a borrar\Archivos Viejos\,Observaciones.xlsm,2020-01-23 12:10:12.440,NaT,None,None,None
2,42,Z:\....Candidatos a borrar\Archivos Viejos\TODO\,IMPRESION ROTULOS DEPOSITO.xls,2020-01-23 12:10:12.707,NaT,None,None,None
3,43,Z:\....Candidatos a borrar\Carpetas compartidas\,primeros pedidos por zona 3.xlsx,2020-01-23 12:10:12.833,NaT,None,None,None
4,44,Z:\....Candidatos a borrar\consultas\movimientos\,Faltannu por fecha.xls,2020-01-23 12:10:12.963,NaT,None,None,None


In [28]:
ID name	query	str_cnn	parent_name	Address_0

119

In [ ]:
cursor = cnxn.cursor()
for index,row in df.iterrows():                                                          
    cursor.execute("INSERT INTO tabla(,,,]) values (?,?,?)",
                   row['col'])
    cnxn.commit()
cursor.close()
cnxn.close()


datos = []



In [1]:
#PROBANDO CON SQLALCHEMY
from sqlalchemy import create_engine
import urllib
server = 'tcp:192.168.1.6'
database = 'planeamiento'
username = 'sa'
password = 'sa'

params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER='+
                                 server+';DATABASE='+database+';UID='+username+';PWD='+ password)

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [2]:
EXCEL_ID = 5
NAME ='PROBANDO NAME'
STR_CONN = 'CONEXION PROBANDO'
SQL = 'SELECT * FROM PRUEBA'

In [3]:
parametros = (EXCEL_ID,NAME,STR_CONN,SQL)
query = "EXEC planeamiento..ctrl_excel_InsertFileConnection @excel_id = %s, @name = '%s', @str_conn = '%s', @sql = '%s'"%parametros

In [5]:
import pandas as pd

In [6]:
#parametros = (EXCEL_ID,NAME,STR_CONN,SQL)
    
#query = "EXEC planeamiento..ctrl_excel_InsertFileConnection @excel_id = {0}, @name = {1}, @str_conn = {2}, @sql = {3}".format(params)
    
#query = 'EXEC my_procedure @Param = {0}'.format(my_param)

In [7]:
df = pd.read_sql_query(query, engine)
#*** HASTA SQLALCHEMY

In [8]:
df

,ctrl_excel_connection_id
0,111.0


In [201]:
server = 'tcp:192.168.1.6'
database = 'planeamiento'
username = 'sa'
password = 'sa'

cnn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)


In [206]:
cursor = cnn.cursor()
cpn = []
parametros = EXCEL_ID,NAME,STR_CONN,SQL

query = """EXEC planeamiento..ctrl_excel_InsertFileConnection  %s, '%s', '%s', '%s'"""%parametros

#sql = "{CALL planeamiento..ctrl_excel_InsertFileConnection (?,?,?,?)}"
try:
    #cursor.execute(sql, params)
    cursor.execute(query)
    rows = cursor.fetchall()
    cursor.commit()
    
except Exception as e:
    print('*'*100)
    print(e)

In [204]:
cursor = cnn.cursor()
#sql = 'exec planeamiento..ctrl_excel_InsertFileConnection(?, ?, ?, ?)'
values = EXCEL_ID,NAME,STR_CONN,SQL

sql = """\
DECLARE @RC int;
EXEC @RC = planeamiento..ctrl_excel_InsertFileConnection ?, ?, ?, ?;
SELECT @RC AS rc;
"""

cursor.execute(sql, values)
rc = cursor.fetchval()




In [27]:
df

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
0,40,Z:\....Candidatos a borrar\Archivos Viejos\,LINEA 1118 - ORIGINAL (NO TOCAR).xlsm,2020-01-23 12:10:12.430,2020-01-24 16:13:16.420,None,3A8037B6-12FA-47D6-82D5-5C7B1F2F035D,None
1,41,Z:\....Candidatos a borrar\Archivos Viejos\,Observaciones.xlsm,2020-01-23 12:10:12.440,NaT,None,None,None
2,42,Z:\....Candidatos a borrar\Archivos Viejos\TODO\,IMPRESION ROTULOS DEPOSITO.xls,2020-01-23 12:10:12.707,NaT,None,None,None
3,43,Z:\....Candidatos a borrar\Carpetas compartidas\,primeros pedidos por zona 3.xlsx,2020-01-23 12:10:12.833,NaT,None,None,None
4,44,Z:\....Candidatos a borrar\consultas\movimientos\,Faltannu por fecha.xls,2020-01-23 12:10:12.963,NaT,None,None,None
...,...,...,...,...,...,...,...,...
34654,34695,Z:\Varios\,INTERNOS ACTUALIZADOS 2019.xlsx,2020-01-23 13:18:48.013,NaT,None,None,None
34655,34696,Z:\Varios\,stock en negativos 27 de septiembre.xlsx,2020-01-23 13:18:48.083,NaT,None,None,None
34656,34697,Z:\Ventas\,Libro1.xlsx,2020-01-23 13:18:48.230,NaT,None,None,None
34657,34698,Z:\Ventas\,presentacion 3-6 v2.xlsm,2020-01-23 13:18:48.250,NaT,None,None,None


'Z:\\....Candidatos a borrar\\Archivos Viejos\\TODO\\IMPRESION ROTULOS DEPOSITO.xls'

In [39]:
df['path']

0              Z:\....Candidatos a borrar\Archivos Viejos\
1              Z:\....Candidatos a borrar\Archivos Viejos\
2         Z:\....Candidatos a borrar\Archivos Viejos\TODO\
3         Z:\....Candidatos a borrar\Carpetas compartidas\
4        Z:\....Candidatos a borrar\consultas\movimientos\
5        Z:\....Candidatos a borrar\consultas\movimientos\
6                       Z:\....Candidatos a borrar\varios\
7                       Z:\....Candidatos a borrar\varios\
8                       Z:\....Candidatos a borrar\varios\
9                       Z:\....Candidatos a borrar\varios\
10        Z:\....Candidatos a borrar\varios\dortiz Planif\
11        Z:\....Candidatos a borrar\varios\dortiz Planif\
12        Z:\....Candidatos a borrar\varios\dortiz Planif\
13       Z:\....Candidatos a borrar\varios\dortiz Plani...
14       Z:\....Candidatos a borrar\varios\dortiz Plani...
15       Z:\....Candidatos a borrar\varios\dortiz Plani...
16       Z:\....Candidatos a borrar\varios\dortiz Plani.

In [30]:
print(df.loc[4]['path']+df.loc[4]['filename'])

Z:\....Candidatos a borrar\consultas\movimientos\Faltannu por fecha.xls


In [23]:
a = check_excel(df.loc[2]['path']+df.loc[2]['filename'])

'None'

In [15]:
PPP= df.loc[2]['path']+df.loc[2]['filename']

In [14]:
pd.DataFrame(a)

,0
0,Excel.Application.Workbooks
1,[]


In [75]:
# 1 - creo un df con la query 'select * from planeamiento..ctrl_excel where end_processing_date is null'
# 2 - recorro el df leyendo cada excel en cada vuelta del loop llama a una función que devuelve todos los parametros (cadena de conexion,query, nombre de la query, parametros etc.)
# 3 - cada lista que devuelve la función la inserto en la base de datos. ( llamada a procedures)

# Estos son lo procedures:
#     planeamiento..ctrl_excel_InsertFileConnection (Devuelve un excel_connection_id para insertar en la proxima tabla)
#     planeamiento..ctrl_excel_InsertFileConnectionLocation (Devuelve un excel_connection_location_id para insertar en la prox tabla)
#     planeamiento..ctrl_excel_InsertFileConnectionParameter (Devuelve excel_connection_location_parameter_id)
    
    


In [53]:
import os
from glob import glob
archivos = []
PATH = "C:\\Users\\jhernandez\\Desktop"
EXT = ["*.xls","*.xlsx","*.xlsm"]
files = []
for x in EXT:
    a = [file
                     for path, subdir, files in os.walk(PATH)
                     for file in glob(os.path.join(path, x))]
    files.append(a)

files

[['C:\\Users\\jhernandez\\Desktop\\Calidad IMPRESION ROTULOS.xls',
  'C:\\Users\\jhernandez\\Desktop\\excels\\Calidad IMPRESION ROTULOS.xls'],
 ['C:\\Users\\jhernandez\\Desktop\\050220 AC scrapers.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\archivo de ejemplo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\~$archivo de ejemplo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\conteo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\ejemplo3.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\pruebaMod.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo_modFecha.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo - copia.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo.xlsx',
  'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\carpeta3\\carpetata 4\\050220 AC scrapers.xlsx',
  'C:\\

In [46]:
archivos

['C:\\Users\\jhernandez\\Desktop\\Canjes electros.xlsm',
 'C:\\Users\\jhernandez\\Desktop\\excels\\macro 2.xlsm',
 'C:\\Users\\jhernandez\\Desktop\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\~$archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\conteo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\ejemplo3.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\pruebaMod.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo_modFecha.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo - copia.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\carpeta3\\carpetata 4\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\excels\\130120

In [47]:
import os
from glob import glob
archivos = []
PATH = "C:\\Users\\jhernandez\\Desktop"
EXT = "*.xls"

xls = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
EXT = "*.xlsx"
xlsx = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]

EXT = "*.xlsm"
xlsm = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]

archivos = xls + xlsx +xlsm

In [48]:
list(archivos)

['C:\\Users\\jhernandez\\Desktop\\Calidad IMPRESION ROTULOS.xls',
 'C:\\Users\\jhernandez\\Desktop\\excels\\Calidad IMPRESION ROTULOS.xls',
 'C:\\Users\\jhernandez\\Desktop\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\~$archivo de ejemplo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\conteo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\ejemplo3.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\pruebaMod.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\Scrapers_testeo_modFecha.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo - copia.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\Scrapers_testeo.xlsx',
 'C:\\Users\\jhernandez\\Desktop\\Carrpeta\\carpeta2\\carpeta3\\carpetata 4\\050220 AC scrapers.xlsx',
 'C:\\Users\\jhernande

In [ ]:
for x in archivos:
    cont = 0
    #iteracion por cada x con check_excel
    # if check_excel(x)["parametros"] != null:
        #hacer un insert en: select * from planeamiento..ctrl_excel_connection_parameter 
            #ELSE: cont +1
    # if check_excel(x)["string_connection"]!= null:
        #hacer un insert en:select * from planeamiento..ctrl_excel_connection 
            #ELSE: cont +1
    #if check_excel(x)["text_command"]!= null:
        #hacer un insert en:select * from planeamiento..ctrl_excel_connection_location 
            #ELSE: cont +1
    #if cont > 0:
    #  insertar ese excel en una nueva tabla(excels candidatos a borrar) y describir que parametros le falta F:
    print(x)

In [7]:
import datetime
#datetime.datetime.today()

In [10]:
datos = []
errores = []
start = datetime.datetime.today()
for index,row in df.iterrows():
    
    archivo = row['path']+row['filename']
    result = check_excel(archivo)
    if result[0] != 0:
        
        errores.append([row['excel_id'],archivo,result[0]])
        
            
    else:
        respuesta = result[0]
        if result[1] != []:
            
            if type(result[1]) == list:
                for res in result[1]:
                    res.append(row['excel_id'])
                    res.append(index)
                
            else:
                result[1].append(row['excel_id'])
                result[1].append(index)
            datos += result[1]
            #raise
    if index == 100:raise    
#     respuesta = result[0]
#     result[1]
end = datetime.datetime.today()

IndexError: list index out of range

In [165]:
end = datetime.datetime.today()
print(end-start)

0:01:17.455000


In [171]:
for m in errores:
    if m[0] == 60:
        print(m)
        raise

In [175]:
for m in datos:
    if m[-2] == 60:
        print(m)
        raise

['Consulta desde estimaciones3', 'SELECT procesos_calidad.articulo, procesos_calidad.insumo, procesos_calidad.color, procesos_calidad.descrip, procesos_calidad.clasificacion, procesos_calidad.clase\r\nFROM ESTIMACIONES.dbo.procesos_calidad procesos_calidad\r\nWHERE (procesos_calidad.articulo=?)\r\nORDER BY procesos_calidad.insumo, procesos_calidad.color', 'ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES;UID=estimaciones;PWD=estimaciones;APP=2007 Microsoft Office \r\nsystem;WSID=WS245;DATABASE=ESTIMACIONES;LANGUAGE=us_english', 'PROCESOS', '93620102003', '$D$3', 'PROCESOS', '39153736009', 60, 20]


RuntimeError: No active exception to reraise

In [145]:
row['path']+row['filename']

'Z:\\Consultas\\Calendarios\\en desuso\\Circuitos Zonas.xlsx'

In [155]:
#xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
#file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'
xl = win32com.client.Dispatch('Excel.Application')
file= row['path']+row['filename']
try:
    wb = xl.Workbooks.Open(file, None,True)
except Exception as e:
    print(e)
    

datos = []

<win32com.gen_py.Microsoft Excel 12.0 Object Library.Connections instance at 0x166810456>

In [156]:
for x in wb.Connections:
        for i in x.Ranges:
            try:
                for p in i.ListObject.QueryTable.Parameters:
                    print(x,
                          x.ODBCConnection.CommandText,
                          x.ODBCConnection.Connection,
                          i.Parent.Name,
                          i.Cells(1,1)[2],
                          p.SourceRange.Address,
                          p.SourceRange.Parent.Name,
                          p.SourceRange.Value)

            except Exception as e:
                try:
                    print(x,
                          x.ODBCConnection.CommandText,
                          x.ODBCConnection.Connection,
                          i.Parent.Name,
                          i.Cells(1,1)[2],
                          None,
                          None,
                          None)
                except:
                    try:
                        for p in i.ListObject.QueryTable.Parameters:
                            print(x,
                                  x.OLEDBConnection.CommandText,
                                  x.OLEDBConnection.Connection,
                                  i.Parent.Name,
                                  i.Cells(1,1)[2],
                                  p.SourceRange.Address,
                                  p.SourceRange.Parent.Name,
                                  p.SourceRange.Value)
                    except:
                        print(x,
                                      x.OLEDBConnection.CommandText,
                                      x.OLEDBConnection.Connection,
                                      i.Parent.Name,
                                      i.Cells(1,1)[2],
                                      None,
                                      None,
                                      None)

Consulta desde ESTIMACIONES SELECT vFechasProcesos2.g, vFechasProcesos2.cp, vFechasProcesos2.zona, vFechasProcesos2.segmento, vFechasProcesos2.categoria, vFechasProcesos2.estadoSIS, vFechasProcesos2.sistemaSIS, vFechasProcesos2.fechaCalendario, vFechasProcesos2.fechaReal, vFechasProcesos2.UnidadesIngresadas, vFechasProcesos2.PedidosIngresados, vFechasProcesos2.fechaLiberada, vFechasProcesos2.fechaRetenida, vFechasProcesos2.fechaDesarme, vFechasProcesos2.estadoActual, vFechasProcesos2.despacho72Hs, vFechasProcesos2.despachadoReal, vFechasProcesos2.Retraso, vFechasProcesos2.tipoEnvio, vFechasProcesos2.fechaProceso, vFechasProcesos2.areaResponsable, vFechasProcesos2.estadoProceso, vFechasProcesos2.fechaUltimoIngreso, vFechasProcesos2.fechaBajada, vFechasProcesos2.UltimaActualizacion, vFechasProcesos2.ubicacion, vFechasProcesos2.prioridadfecha, vFechasProcesos2.prioridadtipo, vFechasProcesos2.ComentarioRegional, vFechasProcesos2.ComentarioJB,vFechasProcesos2.catalogos
FROM ESTIMACIONES.dbo

In [151]:
#df = pd.DataFrame(datos)
#df = df.applymap(str)
#datos = map(str,datos)
if type(foo[0]) == list:
    datos = [list(map(str,i))for i in foo]
else:
    datos = list(map(str,foo))

wb.Close(0)
xl.Application.Quit()

#return df
return 0, datos

com_error: (-2147352567, 'Ocurrió una excepción.', (0, None, None, None, 0, -2147467263), None)

In [144]:
index

66